# POSI - Laby C1

Analiza danch z pomiarów

In [148]:
import pandas as pd
import numpy as np
import re
import os
import plotly.graph_objects as go
import plotly.express as px
from glob import glob
from typing import List

In [77]:
class AcordeonData:
    def __init__(self, file_path):
        with open(file_path, 'r') as file:
            line = file.readline()
            match = re.match(r'(\w+)\s+\[(.*),(.*)\]', line)
            self.function = match.group(1)
            self.range_from = float(match.group(2))
            self.range_to = float(match.group(3))
            
            line = file.readline()
            match = re.match(r'iteration:\s+(.*)', line)
            self.iteration = int(match.group(1))
            
            line = file.readline()
            match = re.match(r'imporvisations:\s+(.*)', line)
            self.imporvisations = int(match.group(1))
            
            line = file.readline()
            match = re.match(r'HMS:\s+(.*)', line)
            self.HMS = int(match.group(1))
            
            line = file.readline()
            match = re.match(r'HMCR:\s+(.*)', line)
            self.HMCR = float(match.group(1))
            
            line = file.readline()
            match = re.match(r'Pitch Adjust Radius:\s+(.*)', line)
            self.pitch_adjust_radius = float(match.group(1))
            
            line = file.readline()
            match = re.match(r'Pitch Adjust Ratio\(PAR\):\s+(.*)', line)
            self.PAR = float(match.group(1))
            
            line = file.readline()
            columns = line.split()
            
            self.df = pd.DataFrame([[float(x) for x in l.split()] for l in file.readlines()])
            self.df.columns = columns

## Automate plot generation

In [229]:
def read_data_from_directory(directory: str) -> List[AcordeonData]:
    files = glob(f'{directory}/*.txt')
    
    data = (AcordeonData(file) for file in files)
    return sorted(data, key=lambda d: d.iteration)

In [268]:
def generate_plots_for_one_simulation(simulation_directory: str, function_optimum: float, figures_scale: int = 3) -> None:
    data_directory = f'{simulation_directory}/data'
    results = read_data_from_directory(data_directory)
    data_frames = [r.df for r in results]
    
    figures_directory = f'{simulation_directory}/figures'
    if not os.path.exists(figures_directory):
        os.mkdir(figures_directory)
        
    x = [r.iteration for r in results]
      
    # Distance from optimum plot
    fig = go.Figure()
    fig.update_layout(title='Odległość od minimum',
                       xaxis_title='Iteracja',
                       yaxis_title='Odległość',
                         xaxis = dict(
                            tickmode = 'array',
                            tickvals = x,
                            ticktext = x
                        ),
                      yaxis_type="log"
#                       yaxis = dict(
#                       range=[0, 80]
#                   )
                     )

    x = [r.iteration for r in results]

    best_values = np.array([df.iloc[0,-1] for df in data_frames])
    best_values_diffrences_from_optimum = np.abs(best_values - function_optimum)
    fig.add_trace(go.Scatter(x=x, y=best_values_diffrences_from_optimum,
                        mode='lines+markers',
                        name='Najlepsza harmonia'))

    means = np.array([df.iloc[:,-1].mean() for df in data_frames])
    means_diffrences_from_optimum = np.abs(means - function_optimum)
    fig.add_trace(go.Scatter(x=x, y=means_diffrences_from_optimum,
                        mode='lines+markers',
                        name='Średnia harmonia'))

    worst_values = np.array([df.iloc[-1,-1] for df in data_frames])
    worst_values_diffrences_from_optimum = np.abs(worst_values - function_optimum)
    fig.add_trace(go.Scatter(x=x, y=worst_values_diffrences_from_optimum,
                        mode='lines+markers',
                        name='Najgorsza harmonia'))

    image_path = f'{figures_directory}/Odleglosc_od_min.png'
    fig.write_image(image_path, scale=3)
    fig.show()
    
    # Standard divation figure
    fig = go.Figure()
    fig.update_layout(title='Skupienie harmonii',
                       xaxis_title='Iteracja',
                       yaxis_title='Odchylenie standardowe',
                        xaxis = dict(
                            tickmode = 'array',
                            tickvals = x,
                            ticktext = x
                        ),
                      yaxis_type="log"
#                       yaxis = dict(
#                       range=[0, 14]
#                   )
                     )

    standard_devations = np.array([df.iloc[:,-1].std() for df in data_frames])
    fig.add_trace(go.Scatter(x=x, y=standard_devations,
                        mode='lines+markers',
                        name='Odchylenie standardowe wartości'))
    
    image_path = f'{figures_directory}/Odchylenie_standardowe.png'
    fig.write_image(image_path, scale=3)
    fig.show()
    
    # Box plot dla wszystkich iteracji
    fig = go.Figure()

    fig.update_layout(title='Wykresy pudełkowe dla danej iteracji', 
                      xaxis_title='Iteracja',
                      yaxis_title='Wartość funkcji',
                      yaxis_type="log"
                      )

#     fig.add_shape(
#             # Line Horizontal
#                 type="line",
#                 x0=-1,
#                 y0=function_optimum,
#                 x1=11,
#                 y1=function_optimum,
#                 line=dict(
#                     color="LightSeaGreen",
#                     width=2,
#                     dash="dashdot",
#                 ),
#         )

    for acordeon_data in results:
        name = f'Iteracja {acordeon_data.iteration}'
        df = acordeon_data.df
        fig.add_trace(go.Box(y=df.iloc[:, -1], name=name))

    image_path = f'{figures_directory}/Wykresy_pudelkowe.png'
    fig.write_image(image_path, scale=3)
    fig.show()

## Test plots on Rastragin

In [103]:
optimum = -20

### Ustawienia standardowe

In [83]:
abs(-20 +9)

11

In [117]:
directory = r'./Pomiary/Rastragrin/50;0.90;0.7/data'

results = read_data_from_directory(directory)
data_frames = [r.df for r in results]

df = data_frames[0]

df.iloc[:,-1].mean()

15.005544167146068

In [253]:
fig = go.Figure()

fig.update_layout(title='Odległość od minimum',
                   xaxis_title='Iteracja',
                   yaxis_title='Odległość',
                     xaxis = dict(
                        tickmode = 'array',
                        tickvals = x,
                        ticktext = x
                    ),
                  yaxis = dict(
                      range=[0, 80]
                  )
                 )

x = [r.iteration for r in results]

best_values = np.array([df.iloc[0,-1] for df in data_frames])
best_values_diffrences_from_optimum = np.abs(best_values - optimum)
fig.add_trace(go.Scatter(x=x, y=best_values_diffrences_from_optimum,
                    mode='lines+markers',
                    name='Najlepsza harmonia'))

means = np.array([df.iloc[:,-1].mean() for df in data_frames])
means_diffrences_from_optimum = np.abs(means - optimum)
fig.add_trace(go.Scatter(x=x, y=means_diffrences_from_optimum,
                    mode='lines+markers',
                    name='Średnia harmonia'))

worst_values = np.array([df.iloc[-1,-1] for df in data_frames])
worst_values_diffrences_from_optimum = np.abs(worst_values - optimum)
fig.add_trace(go.Scatter(x=x, y=worst_values_diffrences_from_optimum,
                    mode='lines+markers',
                    name='Najgorsza harmonia'))
    
image_path = f'{figures_path}/Odleglosc_od_min.png'
# fig.write_image(image_path, scale=3)
fig.show()

In [255]:
fig = go.Figure()

fig.update_layout(title='Skupienie harmonii',
                   xaxis_title='Iteracja',
                   yaxis_title='Odchylenie standardowe',
                    xaxis = dict(
                        tickmode = 'array',
                        tickvals = x,
                        ticktext = x
                    ),
                  yaxis = dict(
                      range=[0, 14]
                  )
                 )

standard_devations = np.array([df.iloc[:,-1].std() for df in data_frames])
fig.add_trace(go.Scatter(x=x, y=standard_devations,
                    mode='lines+markers',
                    name='Odchylenie standardowe wartości'))

In [192]:
# Box plot dla wszystkich iteracji
fig = go.Figure()

fig.update_layout(title='Wykresy pudełkowe dla danej iteracji', 
                  xaxis_title='Iteracja',
                  yaxis_title='Wartość funkcji',
                  )

fig.add_shape(
        # Line Horizontal
            type="line",
            x0=-1,
            y0=-20,
            x1=11,
            y1=-20,
            line=dict(
                color="LightSeaGreen",
                width=2,
                dash="dashdot",
            ),
    )

# fig.add_trace(go.Scatter(
#     x=[-1, 11],
#     y=[-20, -20],
#     mode='lines',
#     line = dict(width=2, dash='dash'),
#     name = 'Globalne miminum'
# ))

for acordeon_data in results:
    name = f'Iteracja {acordeon_data.iteration}'
    df = acordeon_data.df
    fig.add_trace(go.Box(y=df.iloc[:, -1], name=name))
    
fig.show()

In [147]:
# Box plot dla pierwszej iteracji
fig = go.Figure()
df = data_frames[0]

fig.update_layout(title='Wykres pudełkowy pierwszej iteracji')
fig.add_trace(go.Box(y=df.iloc[:, -1], name='Iteracja 0'))

In [146]:
# Box plot dla ostatniej iteracji
fig = go.Figure()
df = data_frames[-1]

fig.update_layout(title='Wykres pudełkowy ostatniej iteracji')
fig.add_trace(go.Box(y=df.iloc[:, -1], name='Iteracja 499'))

## Processing

## Rastragin

In [199]:
optimum = -AssertionError20

### 50;0.90;0.7

In [239]:
directory = r'./Pomiary/Rastragrin/50;0.90;0.7'
rastragin_50_90_70 = read_data_from_directory(f'{directory}/data')

# Generate plots
generate_plots_for_one_simulation(directory, optimum)

### 20;0.9;0.7

In [240]:
directory = r'./Pomiary/Rastragrin/20;0.9;0.7'
rastragin_20_90_70 = read_data_from_directory(f'{directory}/data')

# Generate plots
generate_plots_for_one_simulation(directory, optimum)

### 20;1;0

In [241]:
directory = r'./Pomiary/Rastragrin/20;1;0'
rastragin_20_100_0 = read_data_from_directory(f'{directory}/data')

# Generate plots
generate_plots_for_one_simulation(directory, optimum)

### 50;0.45;0.7

In [245]:
directory = r'./Pomiary/Rastragrin/50;0.45;0.7'
rastragin_50_45_70 = read_data_from_directory(f'{directory}/data')

# Generate plots
generate_plots_for_one_simulation(directory, optimum)

### 50;1;0

In [247]:
directory = r'./Pomiary/Rastragrin/50;1;0'
rastragin_50_100_0 = read_data_from_directory(f'{directory}/data')

# Generate plots
generate_plots_for_one_simulation(directory, optimum)

### 100;0.9;0.7

In [249]:
directory = r'./Pomiary/Rastragrin/100;0.9;0.7'
rastragin_100_90_70 = read_data_from_directory(f'{directory}/data')

# Generate plots
generate_plots_for_one_simulation(directory, optimum)

## Rosenbrock

In [ ]:
optimum = 0

### 50;0.45;0.7

In [269]:
directory = r'./Pomiary/Rosenbrock/50;0.45;0.7'
rastragin_50_90_70 = read_data_from_directory(f'{directory}/data')

# Generate plots
generate_plots_for_one_simulation(directory, optimum)

### 50;95;0

In [271]:
directory = r'./Pomiary/Rosenbrock/50;95;0'
rastragin_50_90_70 = read_data_from_directory(f'{directory}/data')

# Generate plots
generate_plots_for_one_simulation(directory, optimum)

### 50;95;0.7

In [272]:
directory = r'./Pomiary/Rosenbrock/50;95;0.7'
rastragin_50_90_70 = read_data_from_directory(f'{directory}/data')

# Generate plots
generate_plots_for_one_simulation(directory, optimum)

### 50;95;1

In [274]:
directory = r'./Pomiary/Rosenbrock/50;95;1'
rastragin_50_90_70 = read_data_from_directory(f'{directory}/data')

# Generate plots
generate_plots_for_one_simulation(directory, optimum)

### 500;0.95;0.7

In [275]:
directory = r'./Pomiary/Rosenbrock/500;0.95;0.7'
rastragin_50_90_70 = read_data_from_directory(f'{directory}/data')

# Generate plots
generate_plots_for_one_simulation(directory, optimum)